In [1]:
import time
from datetime import datetime
from math import *
def getfare(origin, destination, begin_time, end_time):
    def getTimeResult(currentTime):
        parts = currentTime.split(' ')
    #     print(parts)
        time_now = parts[0] + ' ' + parts[1]
        result = datetime.strptime(time_now, '%Y/%m/%d %H:%M:%S')
        return result

    def convert_timedelta(duration):
        days, seconds = duration.days, duration.seconds
        hours = days * 24 + seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = (seconds % 60)
        return hours, minutes, seconds

    def getCurrentHour(currentTime):
        result = getTimeResult(currentTime)
        hour = result.hour
        return hour

    import math

    def getDistance(origin, destination):
        lon1, lat1 = origin
        lon2, lat2 = destination
        radius = 6371 # km

        dlat = math.radians(lat2-lat1)
        dlon = math.radians(lon2-lon1)
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
            * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = radius * c

        return d

    def getTimeIntervalInMinutes(begin_time, end_time):
        result_begin = getTimeResult(begin_time)
        result_end = getTimeResult(end_time)
        delta = result_end - result_begin
        hours, minutes, seconds = convert_timedelta(delta)
        mins = hours * 60 + minutes
        return mins

    # taxi driver pricing model
    resource = "https://opendata.sz.gov.cn/data/dataSet/toDataDetails/29200_04303267"
    # start_price 10 yuan/ 2 kilometer
    # kilo_price 2.6 yuan / kilometer
    # min_price 0.8 yuan / min
    # For the part exceed 25 kilometer, add 30% of the kilo_price
    # For the part exceed 50 kilometer, add 60% of the kilo_price

    # for time later than 23, add 30% to kilo_price and start price
    def get_price(origin, destination, begin_time, end_time):
        time_interval = getTimeIntervalInMinutes(begin_time, end_time)
        currentHour = getCurrentHour(begin_time)
        real_distance = getDistance(origin, destination)
        final_price = 0
        min_price = 0.8 * time_interval

#         print(time_interval, currentHour, real_distance, min_price)
        if real_distance < 2:
            final_price = 10 + min_price
        elif real_distance > 25:
            final_price = 10 + (25 - 2) * 2.6 + (real_distance - 25) * 1.3 * 2.6 + min_price
        elif real_distance > 50:
            final_price = 10 + (25 - 2) * 2.6 + 25 * 1.3 * 2.6 + (real_distance - 50) * 1.6 * 2.6 + min_price
        else:
            final_price = 10 + (real_distance - 2) * 2.6
        if currentHour > 23:
            final_price *= 1.3
        return final_price
    return get_price(origin, destination, begin_time, end_time)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
filenames = os.listdir('./track_exp/')

In [4]:
class Trip:
    def __init__(self,start_time, end_time, start_cord, end_cord, start_idx, end_idx, avg_speed, fare):
        self.start_time = start_time
        self.end_time = end_time
        self.start_cord = start_cord
        self.end_cord = end_cord
        self.distance = self.compute_distance()
        self.start_idx = start_idx
        self.end_idx = end_idx
        self.avg_speed = avg_speed
        self.fare = fare
        
    def compute_distance(self):
        arr1 = self.start_cord
        arr2 = self.end_cord
        side1 = arr1[0] - arr2[0]
        side2 = arr1[1] - arr2[1]
        distance = (side1**2+side2**2)**0.5
        return distance
        

In [5]:
for file_name in filenames:
    car_name = file_name[:-3]
    try:
        df = pd.read_csv('./track_exp/%s'%file_name)
        trip_df = pd.DataFrame({
                            'StartTime':[], 
                            'EndTime':[], 
                            'StartJD':[], 
                            'StartWD':[], 
                            'EndJD':[], 
                            'EndWD':[], 
                            'Distance':[], 
                            'Start_idx': [], 
                            'End_idx': [], 
                            'Avg_speed':[],
                            'fare':[]
            })
        loaded_idxs = df[df['status']==1].index.tolist()
        if len(loaded_idxs)==0:continue

        last_idx = loaded_idxs[0]
        start_idx = 0

        for idx in loaded_idxs:
            if idx-last_idx != 1: ### last trip ends at last_idx, new trip begins at idx

                ### generate trip object
                start_row = df.iloc[start_idx]
                end_row = df.iloc[last_idx]

                avg_speed = np.mean(df[(df.index>start_idx) & (df.index<last_idx)]['v'])
                
                fare = getfare([start_row['jd'], start_row['wd']], [end_row['jd'], end_row['wd']], start_row['time'], end_row['time'])
                
#                 print([start_row['jd'], start_row['wd']], [end_row['jd'], end_row['wd']], start_row['time'], end_row['time'], fare)

                trip = Trip(
                        start_row['time'], 
                        end_row['time'], 
                        np.array([start_row['jd'],
                        start_row['wd']]), 
                        np.array([end_row['jd'],
                        end_row['wd']]),
                        start_idx,
                        last_idx,
                        avg_speed,
                        fare
                       )
                
                trip_df.loc[trip_df.shape[0]+1] = {
                    'StartTime':trip.start_time, 
                    'EndTime': trip.end_time, 
                    'StartJD': trip.start_cord[0], 
                    'StartWD': trip.start_cord[1], 
                    'EndJD': trip.end_cord[0],
                    'EndWD': trip.end_cord[1],
                    'Distance': trip.distance,
                    'Start_idx': trip.start_idx,
                    'End_idx': trip.end_idx,
                    'Avg_speed': trip.avg_speed,
                    'fare':fare
                }
            ### end generation

                start_idx = idx
            last_idx = idx
        trip_df.to_csv('./TripData/trip_%s'%file_name)
        
    except:
        if UnicodeDecodeError:
            print(car_name,'This shit is fucked')
            continue
        else:
            print(file_name, 'Unknown error')
            continue
        
    

gps_readme. This shit is fucked
.DS_St This shit is fucked
data_cleaning.ip This shit is fucked
data_tocsv.ip This shit is fucked
